# 1. Memoria con PostgreSQL
https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/

In [1]:
from decouple import config
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langgraph.prebuilt import create_react_agent  # 👈 construye agente ReAct listo
from psycopg_pool import AsyncConnectionPool

DB_HOST = config("DB_HOST")
DB_PORT = config("DB_PORT")
DB_NAME = config("DB_NAME")
DB_USER = config("DB_USER")
DB_PASSWORD = config("DB_PASSWORD")

DB_URI = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}" "?sslmode=disable"

pool = AsyncConnectionPool(
    conninfo=DB_URI,
    max_size=10,
    kwargs={
        "autocommit": True,
        "prepare_threshold": 0,  # evita errores con statements cacheados
    },
)
checkpointer = AsyncPostgresSaver(pool)

In [3]:
checkpointer

# 2. Caching
https://python.langchain.com/docs/integrations/caches/redis_llm_caching/

In [2]:
import os

REDIS_URL = config("REDIS_URL")

Connecting to Redis at: redis://localhost:6379


In [4]:
import time

from langchain.globals import set_llm_cache
from langchain.schema import Generation
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_redis import RedisCache, RedisSemanticCache

In [6]:
# Initialize RedisCache
redis_cache = RedisCache(redis_url=REDIS_URL)

# Set the cache for LangChain to use
set_llm_cache(redis_cache)

# Initialize the language model
llm = OpenAI(temperature=0)


# Function to measure execution time
def timed_completion(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time


# First call (not cached)
prompt = "Que acciones se pueden realizar en un proyecto de IA?"
result1, time1 = timed_completion(prompt)
print(f"First call (not cached):\nResult: {result1}\nTime: {time1:.2f} seconds\n")

# Second call (should be cached)
result2, time2 = timed_completion(prompt)
print(f"Second call (cached):\nResult: {result2}\nTime: {time2:.2f} seconds\n")

print(f"Speed improvement: {time1 / time2:.2f}x faster")

# Clear the cache
redis_cache.clear()
print("Cache cleared")

03:44:01 httpx INFO   HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
First call (not cached):
Result: 

1. Definir el objetivo del proyecto: El primer paso en un proyecto de IA es definir claramente el objetivo que se quiere lograr. Esto ayudará a enfocar los esfuerzos y recursos en la dirección correcta.

2. Recopilar y limpiar datos: La IA se basa en datos, por lo que es importante recopilar y limpiar datos relevantes y de alta calidad para el proyecto. Esto puede incluir datos estructurados (como bases de datos) y no estructurados (como imágenes o texto).

3. Seleccionar algoritmos y modelos: Una vez que se tienen los datos, se deben seleccionar los algoritmos y modelos adecuados para el proyecto. Esto dependerá del tipo de problema que se esté abordando y de los datos disponibles.

4. Entrenar y validar el modelo: El siguiente paso es entrenar el modelo seleccionado con los datos recopilados. Esto implica ajustar los parámetros del modelo para que pueda 

# Semantic cache

In [ ]:
# Initialize RedisSemanticCache
embeddings = OpenAIEmbeddings()
semantic_cache = RedisSemanticCache(
    redis_url=REDIS_URL, embeddings=embeddings, distance_threshold=0.2
)

# Set the cache for LangChain to use
set_llm_cache(semantic_cache)


# Function to test semantic cache
def test_semantic_cache(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time


# Original query
original_prompt = "Cual es la capital del Peru?"
result1, time1 = test_semantic_cache(original_prompt)
print(
    f"Original query:\nPrompt: {original_prompt}\nResult: {result1}\nTime: {time1:.2f} seconds\n"
)

# Semantically similar query
similar_prompt = "Puedes decirme una ciudad del Peru?"
result2, time2 = test_semantic_cache(similar_prompt)
print(
    f"Similar query:\nPrompt: {similar_prompt}\nResult: {result2}\nTime: {time2:.2f} seconds\n"
)

print(f"Speed improvement: {time1 / time2:.2f}x faster")

# Clear the semantic cache
semantic_cache.clear()
print("Semantic cache cleared")

03:47:09 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
03:47:09 redisvl.index.index INFO   Index already exists, not overwriting.
03:47:10 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
03:47:11 httpx INFO   HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
03:47:12 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Original query:
Prompt: Cual es la capital del Peru?
Result: 

La capital del Perú es Lima.
Time: 3.21 seconds

03:47:13 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Similar query:
Prompt: Puedes decirme una ciudad del Peru?
Result: 

La capital del Perú es Lima.
Time: 0.64 seconds

Speed improvement: 5.00x faster
Semantic cache cleared
